In [48]:
import os, json
import pandas as pd
import random
import numpy as np
from collections import defaultdict, Counter


In [49]:
data = pd.read_json("sample_sfu_combined.jsonl", lines=True)
print(len(data))
duplicates = data[data.duplicated('sentence_text', keep=False)]
duplicates

600


,sentence_text,label


## Prepare new files

In [50]:
random.seed(42)

In [58]:
df = data.sample(frac=1, random_state=42).reset_index(drop=True)

overlap_sentences = df.iloc[:60]
unique_sentences = df.iloc[60:]

for i in range(3):
    overlap = overlap_sentences.iloc[i*20:(i+1)*20]
    unique = unique_sentences.iloc[i*180:(i+1)*180]
    session = pd.concat([overlap, unique]).sample(frac=1, random_state=42+i)
    session.to_json(f'annotation_session_{i+1}.jsonl', orient='records', lines=True)

sessions = []
for i in range(3):
    s = pd.read_json(f'annotation_session_{i+1}.jsonl', lines=True)
    sessions.append(s)

for i, s in enumerate(sessions, 1):
    total = len(s)
    overlap = len(s.merge(overlap_sentences, on='sentence_text'))
    unique = total - overlap
    dupes_within = s.duplicated('sentence_text').sum()
    print(f'Session {i}: total={total}, overlap={overlap}, unique={unique}, duplicates_within={dupes_within}')

overlap_across = sessions[0].merge(sessions[1], on='sentence_text').merge(sessions[2], on='sentence_text')
print(f'Total distinct overlapping sentences across sessions: {len(overlap_across)}')

Session 1: total=200, overlap=20, unique=180, duplicates_within=0
Session 2: total=200, overlap=20, unique=180, duplicates_within=0
Session 3: total=200, overlap=20, unique=180, duplicates_within=0
Total distinct overlapping sentences across sessions: 0
